# Q1


In [ ]:
boards=["A","B","C"]
resources=["labor","mh"]
demands={"A":100,"B":250,"C":300}
mCosts={"A":47,"B":23,"C":62}
bCosts={"A":56,"B":31,"C":72}
rc={("labor","A"):5,("labor","B"):1,("labor","C"):5,
   ("mh","A"):3,("mh","B"):1,("mh","C"):2}
Avail={"labor":2000,"mh":800}

import gurobipy as gp
model=gp.Model()
make=model.addVars(boards,name="make")
buy=model.addVars(boards,name="buy")
model.setObjective(gp.quicksum(mCosts[i]*make[i]+bCosts[i]*buy[i] for i in boards))
model.addConstrs((gp.quicksum(rc[r,b]*make[b] for b in boards) <=Avail[r] for r in resources),name="_")
model.addConstrs(( make[b]+buy[b]==demands[b] for b in boards),name="demand")
model.optimize()
print ( "optimal value: ",model.OBJVAl)
model.printAttr(["X","Obj","SAObjLow","SAObjUP"])
model.printAttr(["RC","LB","SALBLow","SALBUp","UB","SAUBLow","SAUBUp"])
model.printAttr(["Sense","Slack","Pi","RHS","SARHSLow","SARHSUp"])

# Q2

In [ ]:
warehouses=["Tampa","Miami","Fresno"]
markets=["NY","Phil","Chicago","Boston"]
costs={("Tampa","NY"):9,("Tampa","Phil"):14,("Tampa","Chicago"):12,("Tampa","Boston"):17,
          ("Miami","NY"):11,("Miami","Phil"):10,("Miami","Chicago"):6,("Miami","Boston"):10,
          ("Fresno","NY"):12,("Fresno","Phil"):8,("Fresno","Chicago"):15,("Fresno","Boston"):7
          }
supplies={"Tampa":200,"Miami":200,"Fresno":200}
demands={"NY":130,"Phil":170,"Chicago":200,"Boston":150}

model=gp.Model()
x=model.addVars(costs,obj=costs,name=" ")
model.addConstrs((gp.quicksum(x[w,m] for w in warehouses)<=demands[m] for m in markets),name="demand")
model.addConstrs((gp.quicksum(x[w,m] for m in markets)==supplies[w] for w in warehouses),name="supply")
x["Tampa","Boston"].UB=5
x["Fresno","Chicago"].LB=2
x["Miami","Chicago"].UB=0

model.optimize()
print ( "optimal value: ",model.OBJVAl)
model.printAttr(["X","Obj","SAObjLow","SAObjUP"])
model.printAttr(["RC","LB","SALBLow","SALBUp","UB","SAUBLow","SAUBUp"])
model.printAttr(["Sense","Slack","Pi","RHS","SARHSLow","SARHSUp"])

# Q3

In [ ]:
toys=["Cars","Trikes","Scooters"]
resources=["Material","Labor"]
profit={"Cars":26,"Trikes":28,"Scooters":27}
rc={("Labor","Cars"):15,("Labor","Trikes"):20,("Labor","Scooters"):10,
   ("Material","Cars"):20,("Material","Trikes"):10,("Material","Scooters"):15
   }
Avail={"Material":2800,"Labor":40*60}
demands={"Cars":100,"Trikes":70,"Scooters":60}
fixedCosts=500+3600

model=gp.Model()
x=model.addVars(toys,obj=profit,ub=demands,name=" ")
model.modelSense=gp.GRB.MAXIMIZE
model.addConstrs((gp.quicksum(rc[r,t]*x[t] for t in toys)<=Avail[r] for r in resources),name=" ")
model.optimize()
print ( "optimal value including fixed costs: ",model.OBJVAl-fixedCosts)
model.printAttr(["X","Obj","SAObjLow","SAObjUP"])
model.printAttr(["RC","LB","SALBLow","SALBUp","UB","SAUBLow","SAUBUp"])
model.printAttr(["Sense","Slack","Pi","RHS","SARHSLow","SARHSUp"])

# Q4

In [ ]:
import pandas as pd
DobbieLooms=15
PanteraLooms=80
weeks=13
hoursPerWeek=24*7-2
DobbieHours=DobbieLooms*weeks*hoursPerWeek
PanteraHours=PanteraLooms*weeks*hoursPerWeek

carpets=list(range(1,16))
df=pd.read_csv("Q4.csv")

In [ ]:
import math
cost={} 
HourPerYd={}
demand={}
for ca in carpets:
    cost[("Dobbie",ca)]=float(df[df.Carpet==ca]["Dobie(Cost/Yd)"])
    HourPerYd[("Dobbie",ca)]=1./float(df[df.Carpet==ca]["Dobie(Yd/Hr)"]) #from Yd/H to H/Yd
    if not math.isnan(df[df.Carpet==ca]["Pantera(Cost/Yd)"]):
        cost[("Pantera",ca)]=float(df[df.Carpet==ca]["Pantera(Cost/Yd)"])
        HourPerYd[("Pantera",ca)]=1./float(df[df.Carpet==ca]["Pantera(Yd/Hr)"]) #from Yd/H to H/Yd
    cost[("SubContr",ca)]=float(df[df.Carpet==ca]["SubContract(Cost/Yd)"])
    demand[ca]=float(df[df.Carpet==ca]["Demand(Yds)"])
    


In [ ]:
model=gp.Model()
x=model.addVars(cost,obj=cost,name="Prod")
for ca in carpets:
    if ("Pantera",ca)in cost.keys():
        model.addConstr(x["Dobbie",ca]+x["Pantera",ca]+x["SubContr",ca]>=demand[ca],name="Dem"+str(ca))
    else:
        model.addConstr(x["Dobbie",ca]+x["SubContr",ca]>=demand[ca],name="Dem"+str(ca))
model.addConstr(gp.quicksum(HourPerYd[("Dobbie",ca)]*x[("Dobbie",ca)] 
                            for ca in carpets )<=DobbieHours,name="DobbieHours")       
model.addConstr(gp.quicksum(HourPerYd[("Pantera",ca)]*x[("Pantera",ca)] 
                            for ca in carpets if ("Pantera",ca) in HourPerYd.keys())<=PanteraHours,name="PanteraHours")
model.optimize()

In [ ]:
print ( "optimal value : ",model.OBJVAl)
model.printAttr(["X","Obj","SAObjLow","SAObjUP"])
model.printAttr(["RC","LB","SALBLow","SALBUp","UB","SAUBLow","SAUBUp"])
model.printAttr(["Sense","Slack","Pi","RHS","SARHSLow","SARHSUp"])

In [ ]:
#Q2
machineHours=13*(7*24-2)

In [ ]:
-machineHours*-0.64272

In [ ]:
#Q6
outsourced=[ca for ca in carpets if x["SubContr",ca].X>0]
print(outsourced)


In [ ]:
#Q7